In [1]:
import json
import os
from collections import defaultdict
from pathlib import Path
import cv2

In [2]:
def create_unified_mot_dataset():
    # Paths to annotation files
    json_paths = {
        'train': "/kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_train_objects_in_water_life_jacket_rm_fixed.json",
        'val': "/kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_val_objects_in_fixed.json", 
        'test': "/kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_test_objects_in_water_fixed.json"
    }
    
    # Load all datasets
    datasets = {}
    for split, path in json_paths.items():
        print(f"Loading {split}: {path}")
        if os.path.exists(path):
            try:
                with open(path, 'r') as f:
                    datasets[split] = json.load(f)
                print(f"  ✓ Loaded {split}: {len(datasets[split].get('images', []))} images, {len(datasets[split].get('annotations', []))} annotations")
            except Exception as e:
                print(f"  ✗ Error loading {split}: {e}")
                datasets[split] = {'images': [], 'annotations': []}
        else:
            print(f"  ⚠ Warning: {path} not found")
            datasets[split] = {'images': [], 'annotations': []}
    
    # Create mappings for quick lookup
    img_by_id = {}
    ann_by_img = defaultdict(list)
    video_frames = defaultdict(list)
    
    # Process all splits
    for split, data in datasets.items():
        print(f"\nProcessing {split} split...")
        
        # Safely get images
        images = data.get('images', [])
        if images is None:
            print(f"  Warning: 'images' is None in {split}, using empty list")
            images = []
        
        # Index images by ID
        for img in images:
            if isinstance(img, dict) and 'id' in img:
                img_id = img['id']
                img_by_id[img_id] = img
                video_frames[img['video_id']].append(img)
            else:
                print(f"  Warning: Invalid image entry in {split}: {img}")
        
        # Safely get annotations
        annotations = data.get('annotations', [])
        if annotations is None:
            print(f"  Warning: 'annotations' is None in {split}, using empty list")
            annotations = []
        
        # Index annotations by image_id
        for ann in annotations:
            if isinstance(ann, dict) and 'image_id' in ann:
                ann_by_img[ann['image_id']].append(ann)
            else:
                print(f"  Warning: Invalid annotation in {split}: {ann}")
    
    print(f"\nFound {len(video_frames)} unique video IDs")
    
    # Get all unique video IDs
    all_video_ids = sorted(video_frames.keys())
    
    # Create unified structure
    unified_data = {
        'info': datasets.get('train', {}).get('info', {}),
        'licenses': datasets.get('train', {}).get('licenses', []),
        'categories': datasets.get('train', {}).get('categories', []),
        'videos': [],
        'images': [],
        'annotations': []
    }
    
    global_image_id = 0
    global_ann_id = 0
    
    for video_id in all_video_ids:
        print(f"Processing video {video_id}...")
        
        # Create video entry
        video_entry = {
            'id': int(video_id),
            'name': f"video_{video_id}",
            'width': 3840,  # From sample data
            'height': 2160  # From sample data
        }
        unified_data['videos'].append(video_entry)
        
        # Get all frames for this video, sorted by frame_index
        frames = video_frames[video_id]
        frames.sort(key=lambda x: x.get('frame_index', 0))
        
        video_frame_count = 0
        video_ann_count = 0
        
        for frame in frames:
            # Create new global image ID to avoid conflicts
            new_img_id = global_image_id
            global_image_id += 1
            
            # Copy frame info with updated ID
            new_frame = frame.copy()
            new_frame['id'] = new_img_id
            new_frame['video_id'] = int(video_id)
            unified_data['images'].append(new_frame)
            video_frame_count += 1
            
            # Get annotations for this frame (using original image ID)
            orig_img_id = frame['id']
            orig_ann_list = ann_by_img.get(orig_img_id, [])
            
            for orig_ann in orig_ann_list:
                new_ann_id_local = global_ann_id
                global_ann_id += 1
                
                # Create new annotation
                new_ann = {
                    'id': new_ann_id_local,
                    'image_id': new_img_id,
                    'video_id': int(video_id),
                    'category_id': orig_ann.get('category_id'),
                    'track_id': orig_ann.get('track_id'),
                    'bbox': orig_ann.get('bbox'),
                    'area': orig_ann.get('area', 0),
                    'iscrowd': orig_ann.get('iscrowd', 0)
                }
                
                unified_data['annotations'].append(new_ann)
                video_ann_count += 1
        
        print(f"  Video {video_id}: {video_frame_count} frames, {video_ann_count} annotations")
    
    # Save unified dataset
    output_path = "/kaggle/working/unified_mot_dataset.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(unified_data, f, indent=2)
    
    print(f"\n✓ Unified MOT dataset saved to: {output_path}")
    print(f"Total videos: {len(unified_data['videos'])}")
    print(f"Total images: {len(unified_data['images'])}")
    print(f"Total annotations: {len(unified_data['annotations'])}")
    
    return unified_data

In [3]:
# Run the function
if __name__ == "__main__":
    unified_dataset = create_unified_mot_dataset()

Loading train: /kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_train_objects_in_water_life_jacket_rm_fixed.json
  ✓ Loaded train: 27259 images, 159454 annotations
Loading val: /kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_val_objects_in_fixed.json
  ✓ Loaded val: 8584 images, 47678 annotations
Loading test: /kaggle/input/seadronesseemot-annotation/annotations/annotations/instances_test_objects_in_water_fixed.json
  ✗ Error loading test: object of type 'NoneType' has no len()

Processing train split...

Processing val split...

Processing test split...

Found 21 unique video IDs
Processing video 0...
  Video 0: 410 frames, 1856 annotations
Processing video 1...
  Video 1: 715 frames, 10072 annotations
Processing video 2...
  Video 2: 715 frames, 3550 annotations
Processing video 4...
  Video 4: 3490 frames, 22934 annotations
Processing video 5...
  Video 5: 715 frames, 6435 annotations
Processing video 6...
  Video 6: 2098 frames, 

In [4]:
def create_videos_for_specific_ids(target_video_ids=None, fps=25):
    """
    Create videos for specific video IDs from unified_mot_dataset.json
    
    Args:
        target_video_ids: list of video IDs to process (e.g., [0, 5, 7]), None for all
        fps: frames per second for output videos
    """
    UNIFIED_JSON = "/kaggle/working/unified_mot_dataset.json"
    
    # Separate image folders for each split
    IMAGE_FOLDERS = {
        'train': "/kaggle/input/seadronesseemot-train/train",
        'val': "/kaggle/input/seadronesseemot-validation/val", 
        'test': "/kaggle/input/seadronesseemot-test/test"
    }
    
    OUTPUT_DIR = "/kaggle/working/videos"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Load unified dataset
    with open(UNIFIED_JSON, 'r') as f:
        data = json.load(f)
    
    images = data['images']
    videos = data['videos']
    
    # Group images by original video_id
    images_by_video = defaultdict(list)
    for img in images:
        images_by_video[img['video_id']].append(img)
    
    # Filter video IDs
    if target_video_ids is None:
        target_video_ids = [v['id'] for v in videos]
    else:
        target_video_ids = [int(vid) for vid in target_video_ids]
    
    print(f"Creating videos for video IDs: {target_video_ids}")
    
    successful_videos = 0
    
    for video_id in target_video_ids:
        print(f"\n--- Processing video {video_id} ---")
        
        # Get frames for this video
        frames = images_by_video.get(video_id, [])
        if not frames:
            print(f"  No frames found for video {video_id}, skipping")
            continue
        
        # Sort by frame_index
        frames.sort(key=lambda x: x.get('frame_index', 0))
        print(f"  Found {len(frames)} frames")
        
        # Build frame paths (check all possible split folders)
        frame_paths = []
        missing_frames = 0
        
        for frame in frames:
            file_name = frame['file_name']
            
            # Try to find image in train/val/test folders
            frame_path = None
            for split_name, folder_path in IMAGE_FOLDERS.items():
                candidate_path = os.path.join(folder_path, file_name)
                if os.path.exists(candidate_path):
                    frame_path = candidate_path
                    break
            
            if frame_path and os.path.exists(frame_path):
                frame_paths.append((frame_path, frame['frame_index']))
            else:
                print(f"  WARNING: Missing frame '{file_name}' (frame_index: {frame.get('frame_index', 'N/A')})")
                missing_frames += 1
        
        if missing_frames > 0:
            print(f"  {missing_frames}/{len(frames)} frames missing")
        
        if len(frame_paths) < 2:  # Need at least 2 frames for video
            print(f"  Too few valid frames ({len(frame_paths)}), skipping")
            continue
        
        # Sort valid frames by frame_index
        frame_paths.sort(key=lambda x: x[1])
        
        # Read first frame to get dimensions
        first_frame_path, _ = frame_paths[0]
        first_frame = cv2.imread(first_frame_path)
        if first_frame is None:
            print(f"  Could not read first frame {first_frame_path}, skipping")
            continue
        
        height, width = first_frame.shape[:2]
        print(f"  Video dimensions: {width}x{height}")
        
        # Create video writer
        output_path = os.path.join(OUTPUT_DIR, f"video_{video_id:03d}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Write frames
        valid_frames_written = 0
        for frame_path, frame_idx in frame_paths:
            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"  Failed to read frame at index {frame_idx}")
                continue
            
            # Resize if dimensions don't match
            if frame.shape[0] != height or frame.shape[1] != width:
                frame = cv2.resize(frame, (width, height))
            
            writer.write(frame)
            valid_frames_written += 1
        
        writer.release()
        
        print(f"  ✓ Saved {valid_frames_written} frames to {output_path}")
        successful_videos += 1
    
    print(f"\n🎉 Successfully created {successful_videos}/{len(target_video_ids)} videos")
    print(f"📁 Videos saved in: {OUTPUT_DIR}")

In [5]:
# Usage

# 1. Create videos for specific video IDs
create_videos_for_specific_ids(target_video_ids=[0, 5, 7, 8, 9, 12, 13, 14, 15, 18, 19, 21])

Creating videos for video IDs: [0, 5, 7, 8, 9, 12, 13, 14, 15, 18, 19, 21]

--- Processing video 0 ---
  Found 410 frames
  Video dimensions: 3840x2160
  ✓ Saved 410 frames to /kaggle/working/videos/video_000.mp4

--- Processing video 5 ---
  Found 715 frames
  Video dimensions: 3840x2160
  ✓ Saved 715 frames to /kaggle/working/videos/video_005.mp4

--- Processing video 7 ---
  Found 1001 frames
  Video dimensions: 3840x2160
  ✓ Saved 1001 frames to /kaggle/working/videos/video_007.mp4

--- Processing video 8 ---
  Found 1001 frames
  Video dimensions: 3840x2160
  ✓ Saved 1001 frames to /kaggle/working/videos/video_008.mp4

--- Processing video 9 ---
  Found 3709 frames
  Video dimensions: 3840x2160
  ✓ Saved 3709 frames to /kaggle/working/videos/video_009.mp4

--- Processing video 12 ---
  Found 715 frames
  Video dimensions: 3840x2160
  ✓ Saved 715 frames to /kaggle/working/videos/video_012.mp4

--- Processing video 13 ---
  Found 291 frames
  Video dimensions: 3840x2160
  ✓ Saved 29

In [6]:
!zip -r videos.zip /kaggle/working/videos

  adding: kaggle/working/videos/ (stored 0%)
  adding: kaggle/working/videos/video_009.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_005.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_018.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_015.mp4 (deflated 2%)
  adding: kaggle/working/videos/video_007.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_014.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_000.mp4 (deflated 0%)
  adding: kaggle/working/videos/video_013.mp4 (deflated 0%)
  adding: kaggle/working/videos/video_021.mp4 (deflated 2%)
  adding: kaggle/working/videos/video_012.mp4 (deflated 0%)
  adding: kaggle/working/videos/video_019.mp4 (deflated 1%)
  adding: kaggle/working/videos/video_008.mp4 (deflated 0%)
